**Вариант с ИИ ruGPT-3 от Сбербанка**

In [31]:
from transformers import pipeline

# Загружаем модель ruGPT-3 Medium (от Сбербанка)
chatbot = pipeline(
    "text-generation",
    model="sberbank-ai/rugpt3medium_based_on_gpt2",
    tokenizer="sberbank-ai/rugpt3medium_based_on_gpt2"
)

# База знаний FAQ
hospice_faq = hospice_faq = {
    # Организационные
    "как оформить документы?": "Для оформления документов нужны паспорт, полис ОМС и направление врача. Обратитесь к администратору с 9:00 до 18:00.",
    "график работы": "Хоспис работает круглосуточно. Посещение родственников — с 8:00 до 20:00.",
    "контакты": "Телефон администрации: +7 (XXX) XXX-XX-XX. Экстренная связь: +7 (YYY) YYY-YY-YY.",

    # Медицинские
    "какие обезболивающие используются?": "Мы применяем препараты по протоколу ВОЗ. Конкретный план обезболивания назначает врач.",
    "можно ли получить психологическую помощь?": "Да, наш психолог работает ежедневно. Запишитесь через администратора.",

    # Эмоциональная поддержка (заготовки для сложных случаев)
    "как справиться с болью у ребенка?": "Рекомендуем techniques... [можно добавить ссылки на методики]",
    "что сказать ребенку?": "Важно говорить честно, но бережно... [шаблонные советы]",
}

def get_rugpt_response(user_question):
    prompt = f"""
    Ты — профессиональный психолог, работающий в хосписе для детей.
    Отвечай кратко, деликатно и по делу. Не выдумывай факты.

    Вопрос: {user_question}
    Ответ:
    """
    response = chatbot(prompt, max_length=150)[0]["generated_text"]
    return response.split("Ответ:")[-1].strip()  # Берём только ответ
    response = chatbot(
        prompt,
        max_length=max_length,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.9,
        do_sample=True
    )
    return response[0]["generated_text"]

@app.post("/chat")
async def chat(message: str):
    lower_msg = message.lower()

    # Проверка FAQ
    if lower_msg in hospice_faq:
        return {"response": hospice_faq[lower_msg]}

    # Эмоциональные вопросы
    emotional_keywords = ["как справиться", "поддержка", "совет", "боль", "страх"]
    if any(word in lower_msg for word in emotional_keywords):
        return {"response": get_gpt_response(f"Как поддержать семью с тяжелобольным ребёнком? Вопрос: {message}")}

    return {"response": "Извините, я не понял вопрос. Уточните, пожалуйста."}

Device set to use cpu


In [32]:
# Пример использования
print(get_rugpt_response("когда можно посещать больных"))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


В начале курса лечения.


In [39]:
from transformers import pipeline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 1. Инициализация моделей
chatbot = pipeline("text-generation", model="sberbank-ai/rugpt3medium_based_on_gpt2")
vectorizer = TfidfVectorizer()

# 2. База знаний
hospice_faq = hospice_faq = {
    # Организационные
    "как оформить документы?": "Для оформления документов нужны паспорт, полис ОМС и направление врача. Обратитесь к администратору с 9:00 до 18:00.",
    "график работы": "Хоспис работает круглосуточно. Посещение родственников — с 8:00 до 20:00.",
    "контакты": "Телефон администрации: +7 (XXX) XXX-XX-XX. Экстренная связь: +7 (YYY) YYY-YY-YY.",
    "можно ли находиться с ребенком ночью": "Да, но требуется согласование с дежурным врачом.",

    # Медицинские
    "какие обезболивающие используются?": "Мы применяем препараты по протоколу ВОЗ. Конкретный план обезболивания назначает врач.",
    "можно ли получить психологическую помощь?": "Да, наш психолог работает ежедневно. Запишитесь через администратора.",

    # Эмоциональная поддержка (заготовки для сложных случаев)
    "как справиться с болью у ребенка?": "Рекомендуем techniques... [можно добавить ссылки на методики]",
    "что сказать ребенку?": "Важно говорить честно, но бережно... [шаблонные советы]",
}

# 3. Подготовка векторов FAQ
questions = list(hospice_faq.keys())
X = vectorizer.fit_transform(questions)

def find_similar_question(user_query, threshold=0.7):
    """Находит наиболее похожий вопрос в FAQ"""
    query_vec = vectorizer.transform([user_query])
    similarities = cosine_similarity(query_vec, X)
    max_idx = np.argmax(similarities)
    return questions[max_idx] if similarities[0, max_idx] > threshold else None

def generate_contextual_response(query):
    """Генерация ответа с учётом специфики хосписа"""
    prompt = f"""
    [Контекст: Ты консультант в детском хосписе. Отвечай кратко, профессионально и с сочувствием.]
    Вопрос: {query}
    Ответ:
    """
    response = chatbot(prompt, max_length=150, temperature=0.5)[0]["generated_text"]
    return response.split("Ответ:")[-1].strip()

def process_message(user_query):
    # Шаг 1: Поиск в FAQ
    matched_question = find_similar_question(user_query.lower())
    if matched_question:
        return hospice_faq[matched_question]

    # Шаг 2: Генерация ответа
    return generate_contextual_response(user_query)

Device set to use cpu


In [2]:
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fastapi import FastAPI
import os
# from dotenv import load_dotenv

# Загрузка переменных окружения (если есть)
# load_dotenv()

app = FastAPI()

# 1. Инициализация модели ruGPT-3
print("Загружаю модель ruGPT-3...")
chatbot = pipeline(
    "text-generation",
    model="sberbank-ai/rugpt3medium_based_on_gpt2",
    tokenizer="sberbank-ai/rugpt3medium_based_on_gpt2"
)

# 2. База знаний FAQ
hospice_faq = {

    # Организационные
    "как оформить документы?": "Для оформления документов нужны паспорт, полис ОМС и направление врача. Обратитесь к администратору с 9:00 до 18:00.",
    "график работы": "Хоспис работает круглосуточно. Посещение родственников — с 8:00 до 20:00.",
    "контакты": "Телефон администрации: +7 (XXX) XXX-XX-XX. Экстренная связь: +7 (YYY) YYY-YY-YY.",
    "можно ли находиться с ребенком ночью": "Да, но требуется согласование с дежурным врачом.",

    # Медицинские
    "какие обезболивающие используются?": "Мы применяем препараты по протоколу ВОЗ. Конкретный план обезболивания назначает врач.",
    "можно ли получить психологическую помощь?": "Да, наш психолог работает ежедневно. Запишитесь через администратора.",

    # Эмоциональная поддержка (заготовки для сложных случаев)
    "как справиться с болью у ребенка?": "Рекомендуем techniques... [можно добавить ссылки на методики]",
    "что сказать ребенку?": "Важно говорить честно, но бережно... [шаблонные советы]",
}

# 3. Подготовка системы поиска по FAQ
questions = list(hospice_faq.keys())
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

def find_in_faq(user_query, threshold=0.7):
    """Поиск похожего вопроса в базе FAQ"""
    query_vec = vectorizer.transform([user_query])
    similarities = cosine_similarity(query_vec, X)
    max_idx = np.argmax(similarities)
    return questions[max_idx] if similarities[0, max_idx] > threshold else None

def generate_answer(query):
    """Генерация контекстного ответа"""
    prompt = f"""
    [Контекст: Ты консультант в детском хосписе. Отвечай кратко, профессионально и с сочувствием. Не выдумывай факты.]
    Вопрос: {query}
    Ответ:
    """
    response = chatbot(
        prompt,
        max_length=150,
        temperature=0.5,
        top_k=40,
        do_sample=True
    )
    return response[0]["generated_text"].split("Ответ:")[-1].strip()

@app.post("/chat")
async def chat_handler(user_message: str):
    # 1. Пытаемся найти ответ в FAQ
    matched_question = find_in_faq(user_message.lower())
    if matched_question:
        return {
            "source": "FAQ",
            "answer": hospice_faq[matched_question]
        }

    # 2. Генерируем ответ
    return {
        "source": "ruGPT-3",
        "answer": generate_answer(user_message)
    }

# Для теста в консоли
if __name__ == "__main__":
    from fastapi.testclient import TestClient
    client = TestClient(app)

#     # Тестовые запросы
test_questions = [
    "как подать документы?",
    "можно ли остаться на ночь?",
    "как сказать ребенку о болезни?",
    "какое обезболивание используется?"
    ]

for q in test_questions:
    response = client.post("/chat", json={"user_message": q})
    print(f"Вопрос: {q}")
    # print(f"Ответ ({response.json()['source']}): {response.json()['answer']}\n")
    print(chat_handler(q))

Загружаю модель ruGPT-3...


Device set to use cpu


Вопрос: как подать документы?
<coroutine object chat_handler at 0x7db9d69203c0>
Вопрос: можно ли остаться на ночь?
<coroutine object chat_handler at 0x7db9d69203c0>
Вопрос: как сказать ребенку о болезни?
<coroutine object chat_handler at 0x7db9d69212a0>
Вопрос: какое обезболивание используется?
<coroutine object chat_handler at 0x7db9d6921460>


/tmp/ipython-input-2-2258644503.py:101: RuntimeWarning: coroutine 'chat_handler' was never awaited
  print(chat_handler(q))


In [3]:
# Пример использования
print(chat_handler("когда можно посещать больных"))

<coroutine object chat_handler at 0x7dba6a065620>


/tmp/ipython-input-3-4008307977.py:2: RuntimeWarning: coroutine 'chat_handler' was never awaited
  print(chat_handler("когда можно посещать больных"))


In [11]:
!pip install transformers torch scikit-learn fastapi uvicorn nest-asyncio

from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from fastapi import FastAPI, Request
import nest_asyncio
import uvicorn
from fastapi.testclient import TestClient

# Инициализация модели
print("Загружаю модель ruGPT-3...")
chatbot = pipeline(
    "text-generation",
    model="sberbank-ai/rugpt3medium_based_on_gpt2",
    device="cpu"
)

# База знаний FAQ
hospice_faq = {
    # Организационные
    "как оформить документы?": "Для оформления документов нужны паспорт, полис ОМС и направление врача. Обратитесь к администратору с 9:00 до 18:00.",
    "график работы": "Хоспис работает круглосуточно. Посещение родственников — с 8:00 до 20:00.",
    "контакты": "Телефон администрации: +7 (XXX) XXX-XX-XX. Экстренная связь: +7 (YYY) YYY-YY-YY.",
    "можно ли находиться с ребенком ночью": "Да, но требуется согласование с дежурным врачом.",

    # Медицинские
    "какие обезболивающие используются?": "Мы применяем препараты по протоколу ВОЗ. Конкретный план обезболивания назначает врач.",
    "можно ли получить психологическую помощь?": "Да, наш психолог работает ежедневно. Запишитесь через администратора.",

    # Эмоциональная поддержка (заготовки для сложных случаев)
    "как справиться с болью у ребенка?": "Рекомендуем techniques... [можно добавить ссылки на методики]",
    "что сказать ребенку?": "Важно говорить честно, но бережно... [шаблонные советы]",
}

# Инициализация FastAPI
app = FastAPI()

@app.post("/chat")
async def chat_handler(request: Request):
    data = await request.json()
    user_message = data.get("user_message", "")

    # Поиск в FAQ
    matched_question = None
    for question in hospice_faq:
        if question.lower() in user_message.lower():
            matched_question = question
            break

    if matched_question:
        return {"source": "FAQ", "answer": hospice_faq[matched_question]}

    # Генерация ответа
    prompt = f"Вопрос: {user_message}\nОтвет:"
    response = chatbot(prompt, max_length=100)[0]["generated_text"]
    answer = response.split("Ответ:")[-1].strip()

    return {"source": "ruGPT-3", "answer": answer}

# Тестирование
print("\n=== Тестирование ===")
client = TestClient(app)

test_cases = [
    {"user_message": "как оформить документы?"},
    {"user_message": "Можно ли мне остаться с ребенком на ночь?"},
    {"user_message": "Как сказать ребенку о болезни?"},
    {"user_message": "Что делать при боли?"}
]

for test in test_cases:
    response = client.post("/chat", json=test)
    print(f"Вопрос: {test['user_message']}")
    print(f"Ответ: {response.json()}\n")

# Запуск сервера
print("\n=== Запуск сервера ===")
nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=8000)

ERROR: Operation cancelled by user


KeyboardInterrupt: 